# Generic Rule Induction Notebook

Continuoulsy refined.

In [1]:
#%store -z acc_list 
#%store -z prec_list 

## README

- GLRM needs to run in proper (conda) aix360 environment
- BRCG runs with proper (conda) aix360 environment
- Use aix360i environment for RIPPER

### Configuration


In [2]:
from config import config_dict
from config import config_dict_imbalanced

CONFIG = config_dict_imbalanced['CONFIG-I12']
# CONFIG = config_dict['CONFIG7']
print('Proceed with configuration:', CONFIG['NAME'])
print(CONFIG)

Proceed with configuration: fraud_detection-brcg-trees
{'NAME': 'fraud_detection-brcg-trees', 'DATA_SET': '../data/fraud_detection.csv', 'DATA_TYPES': {}, 'DROP': [], 'MODE': 'PREDICTIVE', 'TRAIN_TEST_SPLIT': 0.3, 'BINARIZER': 'TREES', 'ALGO': 'BRCG', 'TARGET_LABEL': 'Class', 'TYPE': 'BINARY', 'EXAMPLE_FEATURE': 'V6', 'POS_CLASS': 1, 'BASELINE': False, 'ONEHOT': False}


In [3]:
import pandas as pd
import numpy as np
# import os
from sklearn.model_selection import train_test_split #, GridSearchCV
from sklearn.metrics import matthews_corrcoef,fbeta_score,confusion_matrix,f1_score,precision_score, recall_score, accuracy_score, balanced_accuracy_score, confusion_matrix, r2_score, explained_variance_score, mean_absolute_error, max_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from xgboost import XGBClassifier
import time
import warnings
import re

if CONFIG['BINARIZER'] == 'QUANTILE':
    from aix360.algorithms.rbm import FeatureBinarizer
elif CONFIG['BINARIZER'] == 'TREES':
    from aix360.algorithms.rbm import FeatureBinarizerFromTrees
if CONFIG['ALGO'] == 'RIPPER':
    from aix360i.algorithms.rule_induction.ripper import Ripper
elif CONFIG['ALGO'] == 'BRCG':
    from aix360.algorithms.rbm import BooleanRuleCG # BRCGExplainer
elif CONFIG['ALGO'] == 'CORELS':
    from corels import *
elif CONFIG['ALGO'] == 'Witt_RIPPER':
    import wittgenstein as lw
elif CONFIG['ALGO'] == 'GLRM':
    from aix360.algorithms.rbm import GLRMExplainer, LinearRuleRegression


### Data

In [4]:
def convert(char):
    if char == CONFIG['POS_CLASS']:
        return 1
    else:
        return 0

df = pd.read_csv(CONFIG['DATA_SET'],dtype=CONFIG['DATA_TYPES'])
df = df.drop(columns=CONFIG['DROP'])
POS_CLASS = CONFIG['POS_CLASS']
if CONFIG['ALGO'] == 'BRCG': # or CONFIG['ALGO'] == 'CORELS' 
    print('Normalising target label for BRCG')
    df[CONFIG['TARGET_LABEL']] = df[CONFIG['TARGET_LABEL']].map(convert)
    POS_CLASS = 1
    # BRCG trains for value 1 as POS_CLASS
df.info()
df.head()

Normalising target label for BRCG
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
if CONFIG['TYPE'] == 'BINARY':
    target_dist = df[CONFIG['TARGET_LABEL']].value_counts()
    print(target_dist)
    print('Pos label occurrence:', target_dist[POS_CLASS]) 
elif CONFIG['TYPE'] == 'CONTINUOUS':
    df[CONFIG['TARGET_LABEL']].describe()
else:
    print('Unrecognized problem type')

0    284315
1       492
Name: Class, dtype: int64
Pos label occurrence: 492


### Train, Test Split

In [6]:
if CONFIG['TRAIN_TEST_SPLIT'] == 'FIXED':
    if CONFIG['MODE'] == 'PREDICTIVE':
        train = df[df['is_test_set'] == False]
        test = df[df['is_test_set'] == True]
    elif CONFIG['MODE'] == 'DESCRIPTIVE':
        train = df
        test = df

    train = train.drop(columns=['is_test_set'])
    test = test.drop(columns=['is_test_set'])

    y_train = train[CONFIG['TARGET_LABEL']]
    x_train = train.drop(columns=[CONFIG['TARGET_LABEL']])

    y_test = test[CONFIG['TARGET_LABEL']]
    x_test = test.drop(columns=[CONFIG['TARGET_LABEL']])
else:
    x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=[CONFIG['TARGET_LABEL']]), df[CONFIG['TARGET_LABEL']], test_size=CONFIG['TRAIN_TEST_SPLIT'], random_state=42)

print('Training:', x_train.shape, y_train.shape)
print('Test:', x_test.shape, y_test.shape)

Training: (199364, 30) (199364,)
Test: (85443, 30) (85443,)


### Reference Performance

In [7]:
if CONFIG['TYPE'] == 'CONTINUOUS':
    print('needs prior encoding of categoricals')
    # gbr = GradientBoostingRegressor(n_estimators=500, random_state=0)
    # gbr.fit(x_train, y_train)
    # # print('Training R^2:', r2_score(yTrain, gbr.predict(dfTrain)))
    # print('Test R^2:', r2_score(y_test, gbr.predict(x_test)))
elif CONFIG['TYPE'] == 'BINARY':
    x_train_cp = x_train.copy()
    y_train_cp = y_train.copy()
    y_train_cp = y_train.map(convert)  # use manual encoding to make sure that pos_value = 1
    x_test_cp = x_test.copy()
    y_test_cp = y_test.copy()
    y_test_cp = y_test.map(convert)  # use manual encoding to make sure that pos_value = 1
    categorical_features = df.select_dtypes(include=['object']).columns
    # numerical_feat = df.select_dtypes(include=['int64', 'float64']).columns
    for col in categorical_features:
        label_encoder = LabelEncoder()
        label_encoder = label_encoder.fit(df[col])
        x_train_cp[col] = label_encoder.transform(x_train_cp[col])
        x_test_cp[col] = label_encoder.transform(x_test_cp[col])
    xgb_model = XGBClassifier(use_label_encoder=False)
    xgb_model.fit(x_train_cp, y_train_cp)
    y_pred = xgb_model.predict(x_test_cp)

    print('Accuracy:', accuracy_score(y_test_cp, y_pred))
    print('Balanced accuracy:', balanced_accuracy_score(y_test_cp, y_pred))
    print('Precision:', precision_score(y_test_cp, y_pred, pos_label=1))
    print('Recall:', recall_score(y_test_cp, y_pred, pos_label=1))
    # print(xgb_model)



[17:24:10] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Accuracy: 0.9996722961506501
Balanced accuracy: 0.9190824799564756
Precision: 0.95
Recall: 0.8382352941176471


### Binarization

In [8]:
if CONFIG['BINARIZER'] == 'TREES':
    binarizer =  FeatureBinarizerFromTrees(negations=True, randomState=42) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train, y_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)
elif CONFIG['BINARIZER'] == 'QUANTILE':
    binarizer =  FeatureBinarizer(numThresh=9,negations=True) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)  
elif CONFIG['BINARIZER'] == 'NATIVE':
    x_train_bin = x_train
    x_test_bin = x_test
else:
    print('UNRECOGNIZED BINARIZER')

x_train_bin.info() # verbose=True
x_train_bin.head()
#x_train_bin[CONFIG['EXAMPLE_FEATURE']][:10]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 199364 entries, 2557 to 121958
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype
---  ------                --------------   -----
 0   (V10, <=, -1.847304)  199364 non-null  int64
 1   (V10, <=, 2.2171)     199364 non-null  int64
 2   (V10, >, -1.847304)   199364 non-null  int64
 3   (V10, >, 2.2171)      199364 non-null  int64
 4   (V12, <=, -5.436953)  199364 non-null  int64
 5   (V12, >, -5.436953)   199364 non-null  int64
 6   (V14, <=, -8.09768)   199364 non-null  int64
 7   (V14, <=, -4.661454)  199364 non-null  int64
 8   (V14, <=, -2.793189)  199364 non-null  int64
 9   (V14, >, -8.09768)    199364 non-null  int64
 10  (V14, >, -4.661454)   199364 non-null  int64
 11  (V14, >, -2.793189)   199364 non-null  int64
 12  (V17, <=, -2.769057)  199364 non-null  int64
 13  (V17, >, -2.769057)   199364 non-null  int64
 14  (V20, <=, 1.075397)   199364 non-null  int64
 15  (V20, >, 1.075397)    199364 no

feature         V10                                     V12            \
operation        <=                   >                  <=         >   
value     -1.847304  2.217100 -1.847304  2.217100 -5.436953 -5.436953   
2557              0         1         1         0         0         1   
247823            0         1         1         0         0         1   
152342            0         1         1         0         0         1   
103385            0         1         1         0         0         1   
8771              0         1         1         0         0         1   

feature         V14                                                    \
operation        <=                             >                       
value     -8.097680 -4.661454 -2.793189 -8.097680 -4.661454 -2.793189   
2557              0         0         0         1         1         1   
247823            0         0         0         1         1         1   
152342            0         0         0         1         1         1   
103385            0         0         0         1         1         1   
8771              0         0         0         1         1         1   

feature         V17                 V20                 V27            \
operation        <=         >        <=         >        <=         >   
value     -2.769057 -2.769057  1.075397  1.075397  1.694210  1.694210   
2557              0         1         1         0         1         0   
247823            0         1         0         1         1         0   
152342            0         1         1         0         1         0   
103385            0         1         1         0         1         0   
8771              0         1         1         0         1         0   

feature          V3            
operation        <=         >  
value     -3.155919 -3.155919  
2557              0         1  
247823            1         0  
152342            0         1  
103385            0         1  
8771              0         1

### Rule Induction

In [9]:
start_time = time.time()
print('Starting training for', CONFIG['ALGO'])

if CONFIG['ALGO'] == 'BRCG':
    estimator = BooleanRuleCG() # Explainer()
    # estimator.train(x_train, y_train)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        estimator.fit(x_train_bin, y_train)
elif CONFIG['ALGO'] == 'RIPPER':
    estimator = Ripper()
    estimator.fit(x_train_bin, y_train, pos_value=POS_CLASS)
elif CONFIG['ALGO'] == 'Witt_RIPPER':
    estimator = lw.RIPPER()
    estimator.fit(x_train_bin, y_train,class_feat=CONFIG["TARGET_LABEL"] , pos_value=POS_CLASS)
elif CONFIG['ALGO'] == 'GLRM':
    linear_model = LinearRuleRegression() # lambda0=0.0005,lambda1=0.0001
    explainer = GLRMExplainer(linear_model)
    explainer.fit(x_train_bin, y_train)
elif CONFIG['ALGO'] == 'CORELS':
    estimator = CorelsClassifier(n_iter=10000, 
                     max_card=2, # feautres per statement
                     c = 0.0001 # Higher values penalise longer rulelists
                    )
    estimator.fit(x_train_bin, y_train , prediction_name = CONFIG["TARGET_LABEL"])
    
else:
    print('Unrecognized algorithm:', CONFIG['ALGO'])

end_time = time.time()
print('Training time: ' + str(end_time - start_time))

Starting training for BRCG
Learning DNF rule with complexity parameters lambda0=0.001, lambda1=0.001
Initial LP solved
Training time: 0.4759540557861328


### Evaluation

In [10]:
acc_list = []
prec_list = []
%store -r acc_list
%store -r prec_list

if CONFIG['TYPE'] == 'BINARY':
    y_pred = estimator.predict(x_test_bin)
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print('Balanced accuracy:', balanced_accuracy_score(y_test, y_pred))
    print('Precision:', precision_score(y_test, y_pred, pos_label=POS_CLASS))
    print('Recall:', recall_score(y_test, y_pred, pos_label=POS_CLASS))
    print('F1', f1_score(y_test, y_pred, pos_label=POS_CLASS))
    print('ConfusionMatrix', confusion_matrix(y_test, y_pred))
    print('F-2', fbeta_score(y_test, y_pred, pos_label=POS_CLASS, beta= 2))
    print('Mathhews', matthews_corrcoef(y_test, y_pred))

    acc_list.append(recall_score(y_test, y_pred, pos_label=POS_CLASS))
    prec_list.append(precision_score(y_test, y_pred, pos_label=POS_CLASS))
    %store acc_list
    %store prec_list
   
elif CONFIG['TYPE'] == 'CONTINUOUS':
    y_pred = explainer.predict(x_test_bin)
    print(f'R2 Score = {r2_score(y_test, y_pred)}')
    print(f'Explained Variance = {explained_variance_score(y_test, y_pred)}')
    print(f'Mean abs. error = {mean_absolute_error(y_test, y_pred)}')
    print(f'Max error = {max_error(y_test, y_pred)}')
    


Accuracy: 0.9984082955888721
Balanced accuracy: 0.5
Precision: 0.0
Recall: 0.0
F1 0.0


/Users/hvo/opt/miniconda3/envs/aix360/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ConfusionMatrix [[85307     0]
 [  136     0]]
F-2 0.0
Mathhews 0.0
Stored 'acc_list' (list)
Stored 'prec_list' (list)


/Users/hvo/opt/miniconda3/envs/aix360/lib/python3.6/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/Users/hvo/opt/miniconda3/envs/aix360/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
if CONFIG['TYPE'] == 'CONTINUOUS':
    explanation = explainer.explain()
    print(explanation)
elif CONFIG['ALGO'] == 'BRCG':
    model = estimator.explain()
    if not model['isCNF']:
        print('Number of rules:', len(model['rules']))
        print(model['rules'])
elif CONFIG['ALGO'] == 'RIPPER':
    print('Rule count: ' + str(sum([len(rules) for rules in estimator.rule_map.values()])))
    print('Rule set:')
    print(estimator.rule_list_to_pretty_string())
elif CONFIG['ALGO'] == 'CORELS':
    r_length = len(estimator.rl().rules)
    print("Rule Length:", r_length)
    for i in range(len(estimator.rl().rules[0]["antecedents"])):
        an = len(estimator.rl().rules[i]["antecedents"])
        print(f"Antecedents Length Rule {i}:" , an)
elif CONFIG['ALGO'] == 'Witt_RIPPER':
    print("Rule Length:", len(estimator.ruleset_))

# uncomment the following line for a full optimized view of the model as data frame for GLRM rules
# explanation.style

Number of rules: 0
[]
